In [1]:
show raw:read0`test.txt

"Sabqponm"
"abcryxxl"
"accszExk"
"acctuvwj"
"abdefghi"


In [2]:
/ ungroup helps a lot to flatten 2d data into 1d
find:{[m;v]ungroup ([]x:til count m;y:where each m=v)}
show S:find[raw;"S"]
show E:find[raw;"E"]

x y
---
0 0
x y
---
2 5


In [3]:
m:(.Q.a,"SE")!.Q.a,"az"
show height:-97+"i"$m raw

0 0 1 16 15 14 13 12
0 1 2 17 24 23 23 11
0 2 2 18 25 25 23 10
0 2 2 19 20 21 22 9 
0 1 3 4  5  6  7  8 


In [4]:
/ flatten with ungroup
/ composition with ' to get "til each count each"
show hm:ungroup ([]x:til count height;y:('[til;count]'[height]);h:height)

x y h 
------
0 0 0 
0 1 0 
0 2 1 
0 3 16
0 4 15
0 5 14
0 6 13
0 7 12
1 0 0 
1 1 1 
1 2 2 
1 3 17
1 4 24
1 5 23
1 6 23
1 7 11
2 0 0 
2 1 2 
2 2 2 
2 3 18
..


In [5]:
/ prepare for pairwise distances calc
show cmXY:flip `x1`y1`x2`y2!flip (flip hm`x`y) cross flip hm`x`y

x1 y1 x2 y2
-----------
0  0  0  0 
0  0  0  1 
0  0  0  2 
0  0  0  3 
0  0  0  4 
0  0  0  5 
0  0  0  6 
0  0  0  7 
0  0  1  0 
0  0  1  1 
0  0  1  2 
0  0  1  3 
0  0  1  4 
0  0  1  5 
0  0  1  6 
0  0  1  7 
0  0  2  0 
0  0  2  1 
0  0  2  2 
0  0  2  3 
..


In [6]:
/ build sparse connection matrix
cm:lj/[cmXY;(2!`x1`y1`h1 xcol hm;2!`x2`y2`h2 xcol hm)]

/ dist - true if reachable (heights differ in less than 2), false - if not
cm:update dist:2>abs h1-h2 from cm 

/ remove unreachable
cm:delete from cm where not dist

/ now use 0 for self
cm:update dist:0b from cm where x1=x2,y1=y2

/ delete all non-LRUD-reachable
show cm:delete from cm where 1<abs[x1-x2]+abs y1-y2

cm:delete h1,h2 from cm

x1 y1 x2 y2 h1 h2 dist
----------------------
0  0  0  0  0  0  0   
0  0  0  1  0  0  1   
0  0  1  0  0  0  1   
0  1  0  0  0  0  1   
0  1  0  1  0  0  0   
0  1  0  2  0  1  1   
0  1  1  1  0  1  1   
0  2  0  1  1  0  1   
0  2  0  2  1  1  0   
0  2  1  2  1  2  1   
0  3  0  3  16 16 0   
0  3  0  4  16 15 1   
0  3  1  3  16 17 1   
0  4  0  3  15 16 1   
0  4  0  4  15 15 0   
0  4  0  5  15 14 1   
0  5  0  4  14 15 1   
0  5  0  5  14 14 0   
0  5  0  6  14 13 1   
0  6  0  5  13 14 1   
..


In [7]:
/ min-sum bridge - from https://code.kx.com/q/kb/lp/ and https://github.com/KxSystems/kdb/blob/master/sparse.q
minSumBridge:{select min dist+distj by x1,y1,x2,y2 from ej[`xj`yj;`x1`y1`xj`yj`distj xcol x;`xj`yj xcol x]}

In [9]:
/ converge to get pairwise distance matrix
show r:(minSumBridge/) cm

x1 y1 x2 y2| dist
-----------| ----
0  0  0  0 | 0   
0  0  0  1 | 1   
0  0  0  2 | 2   
0  0  0  3 | 19  
0  0  0  4 | 18  
0  0  0  5 | 17  
0  0  0  6 | 16  
0  0  0  7 | 15  
0  0  1  0 | 1   
0  0  1  1 | 2   
0  0  1  2 | 3   
0  0  1  3 | 20  
0  0  1  4 | 29  
0  0  1  5 | 28  
0  0  1  6 | 27  
0  0  1  7 | 14  
0  0  2  0 | 2   
0  0  2  1 | 3   
0  0  2  2 | 4   
0  0  2  3 | 21  
..


In [10]:
/ part 1
select from r where ([]x:x1;y:y1) in S,([]x:x2;y:y2) in E

x1 y1 x2 y2| dist
-----------| ----
0  0  2  5 | 31  


In [14]:
/ part 2
select from r where ([]x:x1;y:y1) in find[m raw;"a"],([]x:x2;y:y2) in E,dist=min dist

x1 y1 x2 y2| dist
-----------| ----
4  0  2  5 | 29  
